In [1]:
#Code based off of https://stackoverflow.com/questions/28384588/twitter-api-get-tweets-with-specific-id

'''
Gets text content for tweet IDs
'''

# standard
from __future__ import print_function
import getopt
import logging
import os
import sys
# import traceback
# third-party: `pip install tweepy`
import tweepy


In [2]:
# Generate your own at https://apps.twitter.com/app
CONSUMER_KEY = 'YVbyH4mM5lXPyHQlOiF3xoTXU'
CONSUMER_SECRET = '5alujJPCSAj8to9ptkZWcfZFiUdtDiKf87VZsPKZDVhttkuCvY'
OAUTH_TOKEN = '42712287-uuIoijoFWcnebEpZBmfoTYpGKoAQwrn4SiZkIQ3S9'
OAUTH_TOKEN_SECRET = '8bJPMnGjLsKRV51LMS8dDlQP3meEIMBVDLjin8rjbgsbb'

In [35]:
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
api = tweepy.API(auth)

# tweet = api.get_status(231225449051729920)
# print(tweet.text)

def clean_tweet(str):
    return str

conversations = [line.rstrip('\n').split('\t') for line in open('data/twitter_ids.tuning.txt')]

pairs = []
for conversation in conversations:
    try:
        conv = []
        for stat in conversation:
            conv.append(clean_tweet(api.get_status(stat).text))
        pairs.append([conv[0],conv[1]])
        pairs.append([conv[1],conv[2]])
        print(pairs[-1])
    except tweepy.error.TweepError as e:
        # print(e)
        continue
print(len(pairs))

['@LucyLovestrukk are they touring in ireland', "@Suzanne__French I don't think so, I think its just England and maybe Scotland :("]
['@xjosboss I believe so... Two years ago right?', "@zay_nuh Omg yes! This is why you're my baby! I didn't expect anyone to get it."]
["@phxazgrl He's not smart enough for that, is he? Lol", "@jgrissom63 no, he's not but it's fun to taunt him!"]
['@Omegakirby he really is', "@PurpleHeartNep Honestly, I'm pretty shocked he hasn't pestered me at all yet. I must be luckier than I thought."]
["@grace11_8 I know. I don't know what's up with him. Unless its a confidence thing.", "@Freddy_Mash Or maybe he's just really lazy now."]
['@KellonLyons hahaha they are soo cool', "@megs_charlotte hahaha they're just long and thin and walk about with their big ears, awwww"]
['@KaiMicha Nice to hear :)', '@AlexPearson_1 how was your Sunday?']
["@MicMusicPassion That language sounds so strange to me. Lol. But it's fun to learn it!", "@XYENSE haha I'm taking it because I ne

KeyboardInterrupt: 

In [ ]:
print(len())

In [1]:
# global logger level is configured in main()
Logger = None

# batch size depends on Twitter limit, 100 at this time
batch_size=100

def get_tweet_id(line):
    '''
    Extracts and returns tweet ID from a line in the input.
    '''
    (tagid,_timestamp,_sandyflag) = line.split('\t')
    (_tag, _search, tweet_id) = tagid.split(':')
    return tweet_id

def get_tweets_single(twapi, idfilepath):
    '''
    Fetches content for tweet IDs in a file one at a time,
    which means a ton of HTTPS requests, so NOT recommended.

    `twapi`: Initialized, authorized API object from Tweepy
    `idfilepath`: Path to file containing IDs
    '''
    # process IDs from the file
    with open(idfilepath, 'rb') as idfile:
        for line in idfile:
            tweet_id = get_tweet_id(line)
            Logger.debug('get_tweets_single: fetching tweet for ID %s', tweet_id)
            try:
                tweet = twapi.get_status(tweet_id)
                print('%s,%s' % (tweet_id, tweet.text.encode('UTF-8')))
            except tweepy.TweepError as te:
                Logger.warn('get_tweets_single: failed to get tweet ID %s: %s', tweet_id, te.message)
                # traceback.print_exc(file=sys.stderr)
        # for
    # with

def get_tweet_list(twapi, idlist):
    '''
    Invokes bulk lookup method.
    Raises an exception if rate limit is exceeded.
    '''
    # fetch as little metadata as possible
    tweets = twapi.statuses_lookup(id_=idlist, include_entities=False, trim_user=True)
    if len(idlist) != len(tweets):
        Logger.warn('get_tweet_list: unexpected response size %d, expected %d', len(tweets), len(idlist))
    for tweet in tweets:
        print('%s,%s' % (tweet.id, tweet.text.encode('UTF-8')))

def get_tweets_bulk(twapi, idfilepath):
    '''
    Fetches content for tweet IDs in a file using bulk request method,
    which vastly reduces number of HTTPS requests compared to above;
    however, it does not warn about IDs that yield no tweet.

    `twapi`: Initialized, authorized API object from Tweepy
    `idfilepath`: Path to file containing IDs
    '''    
    # process IDs from the file
    tweet_ids = list()
    with open(idfilepath, 'rb') as idfile:
        for line in idfile:
            tweet_id = get_tweet_id(line)
            Logger.debug('Enqueing tweet ID %s', tweet_id)
            tweet_ids.append(tweet_id)
            # API limits batch size
            if len(tweet_ids) == batch_size:
                Logger.debug('get_tweets_bulk: fetching batch of size %d', batch_size)
                get_tweet_list(twapi, tweet_ids)
                tweet_ids = list()
    # process remainder
    if len(tweet_ids) > 0:
        Logger.debug('get_tweets_bulk: fetching last batch of size %d', len(tweet_ids))
        get_tweet_list(twapi, tweet_ids)

def usage():
    print('Usage: get_tweets_by_id.py [options] file')
    print('    -s (single) makes one HTTPS request per tweet ID')
    print('    -v (verbose) enables detailed logging')
    sys.exit()

def main(args):
    logging.basicConfig(level=logging.WARN)
    global Logger
    Logger = logging.getLogger('get_tweets_by_id')
    bulk = True
    try:
        opts, args = getopt.getopt(args, 'sv')
    except getopt.GetoptError:
        usage()
    for opt, _optarg in opts:
        if opt in ('-s'):
            bulk = False
        elif opt in ('-v'):
            Logger.setLevel(logging.DEBUG)
            Logger.debug("main: verbose mode on")
        else:
            usage()
    if len(args) != 1:
        usage()
    idfile = args[0]
    if not os.path.isfile(idfile):
        print('Not found or not a file: %s' % idfile, file=sys.stderr)
        usage()

    # connect to twitter
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
    api = tweepy.API(auth)

    # hydrate tweet IDs
    if bulk:
        get_tweets_bulk(api, idfile)
    else:
        get_tweets_single(api, idfile)

if __name__ == '__main__':
    main(sys.argv[1:])

ModuleNotFoundError: No module named 'tweepy'